# Regular Expressions with the re Module

```{note}
Using regular expressions, you specify the rules for the set of possible strings that you want to match; this set might contain English sentences, or e-mail addresses, or TeX commands, or anything you like. You can then ask questions such as “Does this string match the pattern?”, or “Is there a match for the pattern anywhere in this string?”. You can also use REs to modify a string or to split it apart in various ways.<br><br>
The regular expression language is relatively small and restricted, so not all possible string processing tasks can be done using regular expressions. There are also tasks that can be done with regular expressions, but the expressions turn out to be very complicated. In these cases, you may be better off writing Python code to do the processing; while Python code will be slower than an elaborate regular expression, it will also probably be more understandable.
```

## Simple Patterns

We’ll start by learning about the simplest possible regular expressions. Since regular expressions are used to operate on strings, we’ll begin with the most common task: matching characters.

### Matching Characters

Most letters and characters will simply match themselves. For example, the regular expression `test` will match the string `test` exactly.

There are exceptions to this rule; some characters are special metacharacters, and don’t match themselves. Instead, they signal that some out-of-the-ordinary thing should be matched, or they affect other portions of the RE by repeating them or changing their meaning. Here’s a complete list of the metacharacters; their meanings will be discussed later.

```
. ^ $ * + ? { } [ ] \ | ( )
```

The first metacharacters we’ll look at are `[` and `]`. They’re used for specifying a character class, which is a set of characters that you wish to match. Characters can be listed individually, or a range of characters can be indicated by giving two characters and separating them by a '-'. For example, `[abc]` will match any of the characters a, b, or c; this is the same as `[a-c]`, which uses a range to express the same set of characters. If you wanted to match only lowercase letters, your RE would be `[a-z]`.

Metacharacters (except `\`) are not active inside classes. For example, `[akm$]` will match any of the characters 'a', 'k', 'm', or `'$'`; `'$'` is usually a metacharacter, but inside a character class it’s stripped of its special nature.

You can match the characters not listed within the class by complementing the set. This is indicated by including a `'^'` as the first character of the class. For example, `[^5]` will match any character except '5'. If the caret appears elsewhere in a character class, it does not have special meaning. For example: `[5^]` will match either a '5' or a `'^'`.

Perhaps the most important metacharacter is the backslash, `\`. As in Python string literals, the backslash can be followed by various characters to signal various special sequences. It’s also used to escape all the metacharacters so you can still match them in patterns; for example, if you need to match a `[` or `\`, you can precede them with a backslash to remove their special meaning: `\[` or `\\`.

Some of the special sequences beginning with `'\'` represent predefined sets of characters that are often useful, such as the set of digits, the set of letters, or the set of anything that isn’t whitespace. The following list of special sequences isn’t complete:

* `\d` Matches any decimal digit; this is equivalent to the class `[0-9]`.

* `\D` Matches any non-digit character; this is equivalent to the class `[^0-9]`.

* `\s` Matches any whitespace character; this is equivalent to the class `[ \t\n\r\f\v]`.

* `\w` Matches any alphanumeric character; this is equivalent to the class `[a-zA-Z0-9_]`.

* `\W` Matches any non-alphanumeric character; this is equivalent to the class `[^a-zA-Z0-9_]`.

These sequences can be included inside a character class. For example, `[\s,.]` is a character class that will match any whitespace character, or `','` or `'.'`.

The final metacharacter in this section is `.`. It matches anything except a newline character, and there’s an alternate mode (re.DOTALL) where it will match even a newline. `.` is often used where you want to match “any character”.

### Repeating Things

Being able to match varying sets of characters is the first thing regular expressions can do that isn’t already possible with the methods available on strings. Another capability is that you can specify that portions of the RE must be repeated a certain number of times.

The first metacharacter for repeating things that we’ll look at is `*`. `*` doesn’t match the literal character `'*'`; instead, it specifies that the previous character can be matched zero or more times, instead of exactly once.

For example, `ca*t` will match 'ct' (0 'a' characters), 'cat' (1 'a'), 'caaat' (3 'a' characters), and so forth.

Repetitions such as `*` are greedy; when repeating a RE, the matching engine will try to repeat it as many times as possible. If later portions of the pattern don’t match, the matching engine will then back up and try again with fewer repetitions.

A step-by-step example will make this more obvious. Let’s consider the expression `a[bcd]*b`. This matches the letter 'a', zero or more letters from the class `[bcd]`, and finally ends with a 'b'. Now imagine matching this RE against the string 'abcbd'.

| Step | Matched | Explanation                                                                                       |
|------|---------|---------------------------------------------------------------------------------------------------|
| 1    | `a`     | The `a` in the RE matches.                                                                        |
| 2    | `abcbd` | The engine matches `[bcd]*`, going as far as it can, which is to the end of the string.           |
| 3    | Failure | The engine tries to match `b`, but the current position is at the end of the string, so it fails. |
| 4    | `abcb`  | Back up, so that `[bcd]*` matches one less character.                                             |
| 5    | Failure | Try `b` again, but the current position is at the last character, which is a `'d'`.               |
| 6    | `abc`   | Back up again, so that `[bcd]*` is only matching `bc`.                                            |
| 6    | `abcb`  | Try `b` again. This time the character at the current position is `'b'`, so it succeeds.          |

The end of the RE has now been reached, and it has matched `'abcb'`. This demonstrates how the matching engine goes as far as it can at first, and if no match is found it will then progressively back up and retry the rest of the RE again and again. It will back up until it has tried zero matches for `[bcd]*`, and if that subsequently fails, the engine will conclude that the string doesn’t match the RE at all.

Another repeating metacharacter is `+`, which matches one or more times.

There are two more repeating operators or quantifiers. The question mark character, `?`, matches either once or zero times; you can think of it as marking something as being optional. For example, `home-?brew` matches either 'homebrew' or 'home-brew'.

The most complicated quantifier is `{m,n}`, where m and n are decimal integers. This quantifier means there must be at least m repetitions, and at most n. For example, `a/{1,3}b` will match 'a/b', 'a//b', and 'a///b'. It won’t match 'ab', which has no slashes, or 'a////b', which has four.

You can omit either m or n; in that case, a reasonable value is assumed for the missing value. Omitting m is interpreted as a lower limit of 0, while omitting n results in an upper bound of infinity.

The simplest case `{m}` matches the preceding item exactly m times. For example, `a/{2}b` will only match 'a//b'.

Readers may notice that the three other quantifiers can all be expressed using this notation. `{0,}` is the same as `*`, `{1,}` is equivalent to `+`, and `{0,1}` is the same as `?`. It’s better to use `*`, `+`, or `?` when you can, simply because they’re shorter and easier to read.

## Using Regular Expressions

### Compiling Regular Expressions

Regular expressions are compiled into pattern objects, which have methods for various operations such as searching for pattern matches or performing string substitutions.

In [1]:
import re

p = re.compile('ab*')
p

re.compile(r'ab*', re.UNICODE)

re.compile() also accepts an optional flags argument, used to enable various special features. We’ll go over the available settings later, but for now a single example will do:

In [2]:
p = re.compile('ab*', re.IGNORECASE)

The RE is passed to re.compile() as a string. REs are handled as strings because regular expressions aren’t part of the core Python language, and no special syntax was created for expressing them. The re module is simply a C extension module included with Python, just like the socket or zlib modules.

### The Backslash Plague

As stated earlier, regular expressions use the backslash character (`'\'`) to indicate special forms or to allow special characters to be used without invoking their special meaning. This conflicts with Python’s usage of the same character for the same purpose in string literals.

Let’s say you want to write a RE that matches the string `\section`, you must escape any backslashes and other metacharacters by preceding them with a backslash, resulting in the string `\\section`. The resulting string that must be passed to re.compile() must be `\\section`. However, to express this as a Python string literal, both backslashes must be escaped again. 

| Characters      | Stage                                    |
|-----------------|------------------------------------------|
| `\section`      | Text string to be matched                |
| `\\section`     | Escaped backslash for re.compile()       |
| `"\\\\section"` | Escaped backslashes for a string literal |

In [3]:
print("\\\\section")

\\section


In short, to match a literal backslash, one has to write `'\\\\'` as the RE string. In REs that feature backslashes repeatedly, this leads to lots of repeated backslashes and makes the resulting strings difficult to understand.

The solution is to use Python’s raw string notation for regular expressions; backslashes are not handled in any special way in a string literal prefixed with `'r'`, so `r"\n"` is a two-character string containing `'\'` and `'n'`, while `"\n"` is a one-character string containing a newline.

| Regular String | Raw string   |
|----------------|--------------|
| "ab*"          | r"ab*"       |
| "\\\\section"  | r"\\section" |
| "\\w+\\s+\\1"  | r"\w+\s+\1"  |

### Performing Matches

Once you have an object representing a compiled regular expression, what do you do with it? Pattern objects have several methods and attributes. Only the most significant ones will be covered here.

| Method/Attribute | Purpose                                                                    |
|------------------|----------------------------------------------------------------------------|
| `match()`        | Determine if the RE matches at the beginning of the string.                |
| `search()`       | Scan through a string, looking for any location where this RE matches.     |
| `findall()`      | Find all substrings where the RE matches, and returns them as a list.      |
| `finditer()`     | Find all substrings where the RE matches, and returns them as an iterator. |

`match()` and `search()` return None if no match can be found. If they’re successful, a match object instance is returned, containing information about the match: where it starts and ends, the substring it matched, and more.

In [4]:
p = re.compile('[a-z]+')
p

re.compile(r'[a-z]+', re.UNICODE)

Now, you can try matching various strings against the RE `[a-z]+`. An empty string shouldn’t match at all, since `+` means ‘one or more repetitions’.

In [5]:
# None
p.match("")

Now, let’s try it on a string that it should match, such as tempo. In this case, `match()` will return a match object, so you should store the result in a variable for later use.

In [6]:
m = p.match('tempo_')
m

<re.Match object; span=(0, 5), match='tempo'>

Now you can query the match object for information about the matching string. Match object instances also have several methods and attributes; the most important ones are:

| Method/Attribute | Purpose                                                           |
|------------------|-------------------------------------------------------------------|
| `group()`        | Return the string matched by the RE                               |
| `start()`        | Return the starting position of the match                         |
| `end()`          | Return the ending position of the match                           |
| `span()`         | Return a tuple containing the (start, end) positions of the match |

Trying these methods will soon clarify their meaning:

In [7]:
# greedy
m.group()

'tempo'

In [8]:
m.start(), m.end()

(0, 5)

In [9]:
m.span()

(0, 5)

Since the match() method only checks if the RE matches at the start of a string, start() will always be zero. However, the search() method of patterns scans through the string, so the match may not start at zero in that case.

In [10]:
# None
p.match('::: message--x')

In [11]:
m = p.search('::: message--x')
m

<re.Match object; span=(4, 11), match='message'>

In [12]:
# first greedy
m.group()

'message'

In [13]:
m.span()

(4, 11)

In actual programs, the most common style is to store the match object in a variable, and then check if it was None.

Two pattern methods return all of the matches for a pattern. findall() returns a list of matching strings:

In [14]:
p = re.compile(r'\d+')
p.findall('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')

['12', '11', '10']

`findall()` has to create the entire list before it can be returned as the result. The `finditer()` method returns a sequence of match object instances as an iterator:

In [15]:
iterator = p.finditer('12 drummers drumming, 11 ... 10 ...')
iterator

In [16]:
for match in iterator:
    print(match.span())

(0, 2)
(22, 24)
(29, 31)


### Module-Level Functions

You don’t have to create a pattern object and call its methods; the `re` module also provides top-level functions called `match()`, `search()`, `findall()`, and so forth. 

In [17]:
print(re.match(r'From\s+', 'Fromage amk'))

None


In [18]:
re.match(r'From\s+', 'From amk Thu May 14 19:12:10 1998')

<re.Match object; span=(0, 5), match='From '>

### Compilation Flags

Compilation flags let you modify some aspects of how regular expressions work. Flags are available in the re module under two names, a long name such as `IGNORECASE` and a short, one-letter form such as `I`.

Here’s a table of some available flags.

| Flag          | Meaning                                                                                                     |
|---------------|-------------------------------------------------------------------------------------------------------------|
| ASCII, A      | Makes several escapes like `\w`, `\s` and `\d` match only on ASCII characters with the respective property. |
| DOTALL, S     | Make . match any character, including newlines.                                                             |
| IGNORECASE, I | Do case-insensitive matches.                                                                                |

## More Pattern Power

So far we’ve only covered a part of the features of regular expressions. In this section, we’ll cover some new metacharacters, and how to use groups to retrieve portions of the text that was matched.

### More Metacharacters

* `|` The “or” operator. If A and B are regular expressions, A|B will match any string that matches either A or B.

* `^` Matches at the beginning of lines. Unless the MULTILINE flag has been set, this will only match at the beginning of the string.

In [19]:
p1 = re.compile('^From', re.MULTILINE)
p2 = re.compile('^From')

In [20]:
p1.search('Reciting \nFrom Memory\nFrom other')

<re.Match object; span=(10, 14), match='From'>

In [21]:
p2.search('Reciting \nFrom Memory\nFrom other')

In [22]:
re.search('^From', 'From Here to Eternity')

<re.Match object; span=(0, 4), match='From'>

* `$` Matches at the end of a line, which is defined as either the end of the string, or any location followed by a newline character.



In [23]:
re.search('}$', '{block}')

<re.Match object; span=(6, 7), match='}'>

In [24]:
re.search('}$', '{block} ')

In [25]:
re.search('}$', '{block}\n')

<re.Match object; span=(6, 7), match='}'>

In [26]:
re.search('}$', '{block}\n123')

* `\A` Matches only at the start of the string. When not in MULTILINE mode, `\A` and `^` are effectively the same. In MULTILINE mode, they’re different: `\A` still matches only at the beginning of the string, but `^` may match at any location inside the string that follows a newline character.

* `\Z` Matches only at the end of the string.

* `\b` Word boundary. This is a zero-width assertion that matches only at the beginning or end of a word. A word is defined as a sequence of alphanumeric characters, so the end of a word is indicated by whitespace or a non-alphanumeric character. The following example matches `class` only when it’s a complete word; it won’t match when it’s contained inside another word.

In [27]:
p = re.compile(r'\bclass\b')
p.search('no class at all')

<re.Match object; span=(3, 8), match='class'>

In [28]:
p.search('one subclass is')

* `\B` Another zero-width assertion, this is the opposite of `\b`, only matching when the current position is not at a word boundary.

In [29]:
p = re.compile(r'\Bclass\b')
p.search('no class at all')

In [30]:
p.search('one subclass is')

<re.Match object; span=(7, 12), match='class'>

### Grouping

Frequently you need to obtain more information than just whether the RE matched or not. Regular expressions are often used to dissect strings by writing a RE divided into several subgroups which match different components of interest. 

Groups are marked by the `'('`, `')'` metacharacters.

In [31]:
p = re.compile('(ab)*')
p.match('ababababab')

<re.Match object; span=(0, 10), match='ababababab'>

Groups indicated with `'('`, `')'` capture the starting and ending index of the text that they match. Groups are numbered starting with 0. Group 0 is always present; it’s the whole RE, so match object methods all have group 0 as their default argument.

In [32]:
p = re.compile('(a)b')
m = p.match('ab')
# whole RE
m.group()

'ab'

In [33]:
# whole RE
m.group(0)

'ab'

In [34]:
# RE: a
m.group(1)

'a'

Subgroups are numbered from left to right, from 1 upward. Groups can be nested; to determine the number, just count the opening parenthesis characters, going from left to right.

In [35]:
p = re.compile('(a(b)c)d')
m = p.match('abcd')
# RE: (a(b)c)d
m.group(0)

'abcd'

In [36]:
# a(b)c
m.group(1)

'abc'

In [37]:
# 
m.group(2)

'b'

`group()` can be passed multiple group numbers at a time, in which case it will return a tuple containing the corresponding values for those groups.

In [38]:
m.group(2,1,2)

('b', 'abc', 'b')

In [39]:
# The groups() method returns a tuple containing the strings for all the subgroups, from 1 up to however many there are.
m.groups()

('abc', 'b')

Backreferences in a pattern allow you to specify that the contents of an earlier capturing group must also be found at the current location in the string. For example, `\1` will succeed if the exact contents of group 1 can be found at the current position, and fails otherwise.

In [40]:
# detects doubled words in a string
p = re.compile(r'\b(\w+)\s+\1\b')
p.search('Paris in the the spring').group()

'the the'

## Modifying Strings

Up to this point, we’ve simply performed searches against a static string. Regular expressions are also commonly used to modify strings in various ways, using the following pattern methods:

| Method/Attribute | Purpose                                                                                 |
|------------------|-----------------------------------------------------------------------------------------|
| split()          | Split the string into a list, splitting it wherever the RE matches                      |
| sub()            | Find all substrings where the RE matches, and replace them with a different string      |
| subn()           | Does the same thing as sub(), but returns the new string and the number of replacements |

### Splitting String

The `split()` method is similar to the `split()` method of strings but provides much more generality in the delimiters that you can split by.

In [41]:
p = re.compile(r'\W+')
p.split('This is a test, short and sweet, of split().')

['This', 'is', 'a', 'test', 'short', 'and', 'sweet', 'of', 'split', '']

In [42]:
# limit the number of splits made
p.split('This is a test, short and sweet, of split().', 3)

['This', 'is', 'a', 'test, short and sweet, of split().']

In [43]:
p = re.compile(r'\W+')
# using parentheses, delimiter also returned as part of the list
p2 = re.compile(r'(\W+)')

In [44]:
p.split('This... is a test.')

['This', 'is', 'a', 'test', '']

In [45]:
p2.split('This... is a test.')

['This', '... ', 'is', ' ', 'a', ' ', 'test', '.', '']

### Search and Replace

Another common task is to find all the matches for a pattern, and replace them with a different string. The `sub()` method takes a replacement value, and the string to be processed.

In [46]:
p = re.compile('(blue|white|red)')
p.sub('colour', 'blue socks and red shoes')

'colour socks and colour shoes'

In [47]:
p.sub('colour', 'blue socks and red shoes', count=1)

'colour socks and red shoes'

The `subn()` method does the same work, but returns a 2-tuple containing the new string value and the number of replacements that were performed:

In [48]:
p.subn('colour', 'blue socks and red shoes')

('colour socks and colour shoes', 2)

In [49]:
p.subn('colour', 'no colours at all')

('no colours at all', 0)

Empty matches are replaced only when they’re not adjacent to a previous empty match.

In [50]:
p = re.compile('x*')
p.sub('-', 'abxd')

'-a-b--d-'